In [1]:
import pandas as pd

In [47]:
def city_sdgs_performance(self, data):
    """
    Merge performance (specific for NewTaipei)
    """
    data[['2018', '2019', '2020']] = data[['2018', '2019', '2020']].astype('float').ffill(axis=1).bfill(axis=1)
    # 注意：有分母為0的情況
    # 再修正

    # 以2018年為基準(2018年為0)
    data['2018_rate'] = 0
    data['2019_rate'] = ((data['2019'] - data['2018']) / data['2018']).round(3)
    data['2019_rate'] = data.apply(lambda row: -row['2019_rate'] if row['benchmark'] == '低' else row['2019_rate'], axis=1)
    data['2020_rate'] = ((data['2020'] - data['2019']) / data['2019']).round(3)
    data['2020_rate'] = data.apply(lambda row: -row['2020_rate'] if row['benchmark'] == '低' else row['2020_rate'], axis=1)
    data['id'] = data['id'].astype('str') # transform for merge
    self.city_sdgs = self.city_sdgs.merge(data[['id', 'growth_rate']], on='id', how='inner')


In [48]:
performance_df = pd.read_csv(r'../raw_data/performance/ntp_performance.csv')
city_sdgs_performance(performance_df).to_csv(r'../raw_data/performance/ntp_performance_.csv', index=False)

In [59]:
import plotly.graph_objects as go
def draw_line(data):
    fig = go.Figure()

    for _, d in data.iterrows():
        fig.add_trace(go.Scatter(
            x=[2018, 2019, 2020], 
            y=[d['2018_rate'], d['2019_rate'], d['2020_rate']],
            mode='lines+markers', 
            name=d['id'],
        ))
    
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=[2018, 2019, 2020],
        ),
        template='plotly_dark',
    )

    fig.show()
draw_line(performance_df.iloc[10:30])

In [63]:
import numpy as np
np.linspace(0, 16, 17)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16.])

In [69]:
import plotly.graph_objects as go

from constants import CITY_ABBR
def draw_sankey_multi_cities(main_data, *other_data, sdg_target:str, indicator_target:str = None):
    """
    Args:
        main_data: pd.DataFrame, 主要城市的資料, 在sankey圖的左邊
        other_data: pd.DataFrame, 其他城市的資料, 在sankey圖的右邊
        sdg_target: str, ex: 'SDG11' (required, default: 'SDG11')
        indicator_target: str, ex: '1-1-1' (optional, default: None)
    """

    # 加上city_name縮寫，避免多城市搞混 ex: NTP01
    main_data = main_data.copy()
    main_city = main_data['city_name'].unique()[0]
    main_data['target'] = ' (' + main_data['city_name'].map(CITY_ABBR) + ')' + main_data['target']

    # main_data source 和 target 互換
    main_data['source'], main_data['target'] = main_data['target'], main_data['source']
    selection = main_data[main_data['goal']==sdg_target].sort_values(by='target', ascending=True)


    other_city = []
    for data in other_data:
        data = data.copy()
        other_city.append(data['city_name'].unique()[0])
        # 加上city_name縮寫，避免多城市搞混 ex: TP01, TY01
        data['target'] = ' (' + CITY_ABBR[data['city_name'].unique()[0]] + ')' + data['target']

        if indicator_target:
            data = data[data['source']==f"Indicator{indicator_target}"]
        selection = pd.concat([selection, data[data['goal']==sdg_target].sort_values(by='source', ascending=True)])
    
    node_ids = list(selection['source'].unique()) + list(selection['target'].unique())
    
    def node_id_to_label(node_id):
        """
        get node_id's indicator title for nodes' hover template
        """
        if "Indicator" in node_id:
            return selection.loc[(selection['source'] == node_id) | (selection['target'] == node_id), 'un_indicator'].values[0]
        else:
            return selection.loc[(selection['source'] == node_id) | (selection['target'] == node_id), 'city_indicator'].values[0]

    node_customdata = [node_id_to_label(node_id) for node_id in node_ids]
    # node_customdata = list(selection['un_indicator'].unique()) + list(selection['city_indicator'].unique())
    # 修正：node to node
    # link_customdata = list(selection['value'])

    fig = go.Figure()
    fig.add_trace(go.Sankey(
        node=dict(
            pad=20,
            thickness=20,
            # line= dict(color="black", width=0.5),
            label = node_ids,
            hoverinfo='all',
            customdata=node_customdata,
            hovertemplate='%{label}<br> %{customdata}',
            align='left' if indicator_target else 'justify',
        ),
        link=dict(
            source=[node_ids.index(source) for source in selection['source']],
            target=[node_ids.index(target) for target in selection['target']],
            value=selection['value'],
            # color='blue',
            # customdata = link_customdata,
            # color='blue',
            # customdata = link_customdata,
            hoverinfo='all',
            hovertemplate='source: %{source.label} <br> target: %{target.label} <br> value: %{value} <br> %{customdata}'
        )
    ))

    fig.update_layout(
        title_text=f"Sankey Diagram of {sdg_target} in {main_city} vs {', '.join(other_city)}", # 標題
        margin=dict(l=30, r=30, t=60, b=30), # 圖表邊界
        width=800,
        height=400,
        # style
        # style
        template='plotly_dark',
    )
    fig.show()

In [70]:
ntp_sankey = pd.read_csv('../output/NewTaipei/sankey.csv', index_col=0)
tp_sankey = pd.read_csv('../output/Taipei/sankey.csv', index_col=0)
ty_sankey = pd.read_csv('../output/Taoyuan/sankey.csv', index_col=0)

In [71]:
draw_sankey_multi_cities(ntp_sankey, tp_sankey, ty_sankey, sdg_target='SDG04', indicator_target='4-1-2')
# draw_sankey_multi_cities(ntp_sankey, tp_sankey, ty_sankey, sdg_target='SDG04')